In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
from scipy.stats import bernoulli
from scipy.optimize import fsolve
from scipy.stats import norm
from tabulate import tabulate
%matplotlib inline
import datetime
from datetime import timedelta, date
from dateutil.relativedelta import relativedelta
import math

In [20]:
ytm_raw = pd.read_excel('YTM_시계열 정보.xlsx', sheet_name='2017.05.16_BB-_YTM').set_index('일자') # 시트 이름 바꾸면 됩니다
# 시계열 정보에서 이자율이 표시되지 않은 부분은 열 자체를 삭제해야 작동합니다 ex) 30Y나 50Y의 이자율이 '-'인 경우 str 이어서 처리가 어려움
target_date = "2017-05-16"
mature_date = "2027-05-15"
# 날짜들을 datetime형식으로 가져와서 days, months 처리를 용이하게 만듦
start_date = datetime.datetime.strptime(target_date[2:], "%y-%m-%d").date()
end_date = datetime.datetime.strptime(mature_date[2:], "%y-%m-%d").date()
interval = 4 # 연간 이자 지급 횟수
ytm = ytm_raw.loc[target_date,:] # 해당 날짜의 데이터만 가져옴

In [21]:
df_ytm = ytm.to_frame()
# 시작점 데이터를 맨 앞줄로 넣어줌
df_ytm.loc['0M']=[float(df_ytm.iloc[0][0])]
newIndex=['0M']+[idx for idx in df_ytm.index if idx!='0M']
df_ytm=df_ytm.reindex(index=newIndex)

In [22]:
# 분기별 이자지급을 가정했으므로 분기별로 날짜 정리, 기존 방법 말고 실제 3개월 날짜로 했음
cal_month = []
for i in df_ytm.index:
    if 'M' in i:
        m = i.replace('M','')
        m_int = int(m)
        cal_month.append(m_int)
    if 'Y' in i:
        y = i.replace('Y','')
        y_diff = float(y)
        cal_month.append(round(y_diff*12,0))

df_ytm['month'] = cal_month
print(df_ytm)

      2017-05-16  month
0M         7.321    0.0
3M         7.321    3.0
6M         8.166    6.0
9M         8.917    9.0
1Y         9.557   12.0
1.5Y      10.365   18.0
2Y        11.283   24.0
2.5Y      12.007   30.0
3Y        12.803   36.0
4Y        13.248   48.0
5Y        13.623   60.0
7Y        14.229   84.0
10Y       14.711  120.0
15Y       15.460  180.0
20Y       15.917  240.0


In [23]:
dates = []
for i in df_ytm['month']:
    distance = start_date + relativedelta(months = i)
    dates.append(distance)
df_ytm['date'] = dates
df_ytm = df_ytm.set_index("date")
print(df_ytm)

            2017-05-16  month
date                         
2017-05-16       7.321    0.0
2017-08-16       7.321    3.0
2017-11-16       8.166    6.0
2018-02-16       8.917    9.0
2018-05-16       9.557   12.0
2018-11-16      10.365   18.0
2019-05-16      11.283   24.0
2019-11-16      12.007   30.0
2020-05-16      12.803   36.0
2021-05-16      13.248   48.0
2022-05-16      13.623   60.0
2024-05-16      14.229   84.0
2027-05-16      14.711  120.0
2032-05-16      15.460  180.0
2037-05-16      15.917  240.0


In [24]:
df_ytm['int_diff'] = df_ytm[target_date].diff().fillna(0)
df_ytm['month_diff'] = df_ytm['month'].diff().fillna(0)
print(df_ytm)

            2017-05-16  month  int_diff  month_diff
date                                               
2017-05-16       7.321    0.0     0.000         0.0
2017-08-16       7.321    3.0     0.000         3.0
2017-11-16       8.166    6.0     0.845         3.0
2018-02-16       8.917    9.0     0.751         3.0
2018-05-16       9.557   12.0     0.640         3.0
2018-11-16      10.365   18.0     0.808         6.0
2019-05-16      11.283   24.0     0.918         6.0
2019-11-16      12.007   30.0     0.724         6.0
2020-05-16      12.803   36.0     0.796         6.0
2021-05-16      13.248   48.0     0.445        12.0
2022-05-16      13.623   60.0     0.375        12.0
2024-05-16      14.229   84.0     0.606        24.0
2027-05-16      14.711  120.0     0.482        36.0
2032-05-16      15.460  180.0     0.749        60.0
2037-05-16      15.917  240.0     0.457        60.0


In [27]:
df_ytm['diff_per_month'] = df_ytm['int_diff']/df_ytm['month_diff']
df_ytm = df_ytm.fillna(0)
print(df_ytm)

            2017-05-16  month  int_diff  month_diff  diff_per_month
date                                                               
2017-05-16       7.321    0.0     0.000         0.0        0.000000
2017-08-16       7.321    3.0     0.000         3.0        0.000000
2017-11-16       8.166    6.0     0.845         3.0        0.281667
2018-02-16       8.917    9.0     0.751         3.0        0.250333
2018-05-16       9.557   12.0     0.640         3.0        0.213333
2018-11-16      10.365   18.0     0.808         6.0        0.134667
2019-05-16      11.283   24.0     0.918         6.0        0.153000
2019-11-16      12.007   30.0     0.724         6.0        0.120667
2020-05-16      12.803   36.0     0.796         6.0        0.132667
2021-05-16      13.248   48.0     0.445        12.0        0.037083
2022-05-16      13.623   60.0     0.375        12.0        0.031250
2024-05-16      14.229   84.0     0.606        24.0        0.025250
2027-05-16      14.711  120.0     0.482        3

In [47]:
month_date = []
month_diff = relativedelta(end_date, start_date).months + relativedelta(end_date, start_date).years*12 + 1 # 차이 금액이라서 1 더해야 정확한 구간

for i in range(0, month_diff+1):
    day = start_date + relativedelta(months = i)
    month_date.append(day)

month_ytm = pd.DataFrame({'date': month_date, 'ytm': 0})
month_ytm = month_ytm.set_index('date')
print(month_ytm)
print(df_ytm)

            ytm
date           
2017-05-16    0
2017-06-16    0
2017-07-16    0
2017-08-16    0
2017-09-16    0
...         ...
2027-01-16    0
2027-02-16    0
2027-03-16    0
2027-04-16    0
2027-05-16    0

[121 rows x 1 columns]
            2017-05-16  month  int_diff  month_diff  diff_per_month
date                                                               
2017-05-16       7.321    0.0     0.000         0.0        0.000000
2017-08-16       7.321    3.0     0.000         3.0        0.000000
2017-11-16       8.166    6.0     0.845         3.0        0.281667
2018-02-16       8.917    9.0     0.751         3.0        0.250333
2018-05-16       9.557   12.0     0.640         3.0        0.213333
2018-11-16      10.365   18.0     0.808         6.0        0.134667
2019-05-16      11.283   24.0     0.918         6.0        0.153000
2019-11-16      12.007   30.0     0.724         6.0        0.120667
2020-05-16      12.803   36.0     0.796         6.0        0.132667
2021-05-16      13.2

In [48]:
r_ytm = []
for idx, i in enumerate(month_ytm.index):
    for j in df_ytm.index:
        if i == j:
            r_temp = df_ytm.loc[j][target_date]
            r_ytm.append(r_temp)
            break
        if i < j:
            r_temp = r_ytm[idx-1] + df_ytm.loc[j]['diff_per_month']
            r_ytm.append(r_temp)
            break
        else:
            continue

# print(r_ytm, len(r_ytm))

month_ytm['ytm'] = r_ytm
print(month_ytm)

                  ytm
date                 
2017-05-16   7.321000
2017-06-16   7.321000
2017-07-16   7.321000
2017-08-16   7.321000
2017-09-16   7.602667
...               ...
2027-01-16  14.657444
2027-02-16  14.670833
2027-03-16  14.684222
2027-04-16  14.697611
2027-05-16  14.711000

[121 rows x 1 columns]


In [99]:
r_exp = []
for i in r_ytm:
    r_temp = interval*np.log(1+(i/100)/interval)
    r_exp.append(r_temp)

# print(r_exp)
month_ytm['eytm'] = r_exp
print(month_ytm)

                  ytm      eytm
date                           
2017-05-16   7.321000  0.072548
2017-06-16   7.321000  0.072548
2017-07-16   7.321000  0.072548
2017-08-16   7.321000  0.072548
2017-09-16   7.602667  0.075313
...               ...       ...
2027-01-16  14.657444  0.143953
2027-02-16  14.670833  0.144082
2027-03-16  14.684222  0.144211
2027-04-16  14.697611  0.144340
2027-05-16  14.711000  0.144469

[121 rows x 2 columns]


In [100]:
q_date = []
q_ytm_lst = []

for i in month_ytm.index:
    if relativedelta(i, start_date).months % 3 == 0:
        q_date.append(i)
        q_ytm_lst.append(month_ytm.loc[i,'ytm'])

# print(q_date, len(q_date))
# print(q_ytm_lst, len(q_ytm_lst))

[datetime.date(2017, 5, 16), datetime.date(2017, 8, 16), datetime.date(2017, 11, 16), datetime.date(2018, 2, 16), datetime.date(2018, 5, 16), datetime.date(2018, 8, 16), datetime.date(2018, 11, 16), datetime.date(2019, 2, 16), datetime.date(2019, 5, 16), datetime.date(2019, 8, 16), datetime.date(2019, 11, 16), datetime.date(2020, 2, 16), datetime.date(2020, 5, 16), datetime.date(2020, 8, 16), datetime.date(2020, 11, 16), datetime.date(2021, 2, 16), datetime.date(2021, 5, 16), datetime.date(2021, 8, 16), datetime.date(2021, 11, 16), datetime.date(2022, 2, 16), datetime.date(2022, 5, 16), datetime.date(2022, 8, 16), datetime.date(2022, 11, 16), datetime.date(2023, 2, 16), datetime.date(2023, 5, 16), datetime.date(2023, 8, 16), datetime.date(2023, 11, 16), datetime.date(2024, 2, 16), datetime.date(2024, 5, 16), datetime.date(2024, 8, 16), datetime.date(2024, 11, 16), datetime.date(2025, 2, 16), datetime.date(2025, 5, 16), datetime.date(2025, 8, 16), datetime.date(2025, 11, 16), datetime.d

In [111]:
q_spt = []
sum_ytm = []
temp_sum = 0

for idx, i in enumerate(q_date):  
    if idx == 0:
        q_spt.append(month_ytm.loc[i,'eytm'])
        sum_ytm.append(0)
        continue
#     print("loop: ", idx)
#     print(q_ytm_lst[idx])
#     print(q_date[idx])
#     print((q_date[idx] - start_date).days)
    for j in range(1, idx+1):
#         temp_sum += 1/(1 + q_ytm_lst[idx]/100)**((q_date[j] - start_date).days/365) # 연속복리 아닐 경우
        temp_sum += math.exp(-(q_ytm_lst[idx]/100)*((q_date[j] - start_date).days/365)) # 연속복리
    sum_ytm.append( temp_sum )
    temp_sum = 0
    if idx == 1:
        q_spt.append(month_ytm.loc[i,'eytm'])
    else:
        temp = sum_ytm[idx] - sum_ytm[idx-1] # spot rate 계산을 위한 ytm 할인 쿠폰합(당기 채권가격) - spot 할인 쿠폰합(전기 채권가격)
        l_days = (q_date[idx] - start_date).days # 현재 시기와 발행일자간의 차이
#         q_spt.append( (1/temp)**(365/l_days)-1 ) # 연속복리 아닐 경우
        q_spt.append( -np.log(temp)*(365/l_days) ) # 연속복리
    
#     print("temp: ", temp)
#     print("sum_ytm: ", sum_ytm)
#     print("sum_spt: ",sum_spt, " spt%: ", q_spt)

# print(q_spt)

q_rates = pd.DataFrame({'date': q_date, 'ytm': q_ytm_lst, 'spt': q_spt})
q_rates = q_rates.set_index('date')

print(q_rates)

                  ytm       spt
date                           
2017-05-16   7.321000  0.072548
2017-08-16   7.321000  0.072548
2017-11-16   8.166000  0.085982
2018-02-16   8.917000  0.096944
2018-05-16   9.557000  0.105709
2018-08-16   9.961000  0.108192
2018-11-16  10.365000  0.114507
2019-02-16  10.824000  0.123292
2019-05-16  11.283000  0.130757
2019-08-16  11.645000  0.132799
2019-11-16  12.007000  0.138798
2020-02-16  12.405000  0.147473
2020-05-16  12.803000  0.154417
2020-08-16  12.914250  0.137055
2020-11-16  13.025500  0.138957
2021-02-16  13.136750  0.140889
2021-05-16  13.248000  0.142860
2021-08-16  13.341750  0.142865
2021-11-16  13.435500  0.144560
2022-02-16  13.529250  0.146285
2022-05-16  13.623000  0.148047
2022-08-16  13.698750  0.147150
2022-11-16  13.774500  0.148604
2023-02-16  13.850250  0.150087
2023-05-16  13.926000  0.151601
2023-08-16  14.001750  0.153144
2023-11-16  14.077500  0.154720
2024-02-16  14.153250  0.156332
2024-05-16  14.229000  0.157983
2024-08-